This Python notebook takes a subset of POIs in Downtown Santa Cruz, and moves each one to the closest building.

In [ ]:
!pip install pandas --quiet
!pip install geopandas --quiet
!pip install shapely --quiet
!pip install duckdb --quiet
!pip install jupysql --quiet
!pip install duckdb-engine --quiet
!pip install folium matplotlib mapclassify --quiet

In [ ]:
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt
from shapely import Point
from shapely.ops import nearest_points
import shapely

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb://

In [ ]:
%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql places <<
SELECT
    id,
    names.primary as name,
    ST_AsText(geometry) as geometry
FROM
    read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*')
WHERE
    bbox.xmin BETWEEN -122.027879 AND -122.024587 AND
    bbox.ymin BETWEEN 36.970389 AND 36.978416

In [ ]:
places = gpd.GeoDataFrame(
    places,
    geometry=places['geometry'].apply(wkt.loads),
    crs="EPSG:4326"
)

In [ ]:
%%sql buildings <<
SELECT
    ST_AsText(geometry) as geometry
FROM
    read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=buildings/type=building/*')
WHERE
    bbox.xmin BETWEEN -122.03 AND -122.02 AND
    bbox.ymin BETWEEN 36.97 AND 36.98

In [ ]:
buildings = gpd.GeoDataFrame(
    buildings,
    geometry=buildings['geometry'].apply(wkt.loads),
    crs="EPSG:4326"
)

In [ ]:
places_updated = places.copy()

for i in range(len(places['geometry'])):
    cur_point = places['geometry'][i]
    min_distance = 10000000000000
    min_index = -1
    for j in range(len(buildings['geometry'])):
        cur_building = buildings['geometry'][j]
        if shapely.distance(cur_building, cur_point) < min_distance:
            min_distance = shapely.distance(cur_building, cur_point)
            min_index = j
    if min_index != -1:
        places_updated.at[i, 'geometry'] = nearest_points(buildings['geometry'][min_index], cur_point)[0]

In [ ]:
# Visualize the data

m = buildings.explore()

places.explore(m=m, color="red")
places_updated.explore(m=m, color="green")

m